# Step 1: Cleaning the  Data
In this notebook, we'll take the raw data and reorganize them so that we can use them for visualization and a predictive model. These are the files we will be working with:
```
incidents.csv                 List of gun violence incidents from January 2014 - March 2018
population.csv                Population from 2000 - 2017 intercensal estimates for July
other_crime_annual.csv        Annual sums of crimes by state from 2010 - 2016
income.csv                    Annual average personal income by state from 2009-2017         
annual_gun_deaths.csv         Annual deaths by guns (homicides only) by state from 1999-2013
alcohol.csv                   Annual alcohol consumption by state from 1977-2016
provisions.csv                List of provisions in place by each state for the years 1991-2017
election_results.csv          List of presidential election results by state from years 2000-2016
registrations.csv             List of weapons registrations by state for years 2011-2017
substances.csv                List of substance abuse features by state for years 2011-2016
```
For more information on where these datasets came from, see the writeup. Our goal is to make several DataFrames which will be used later.

In [1]:
# Numpy and pandas for manipulating the data
import numpy as np
import pandas as pd

In [2]:
daily_incidents_file = './data/raw/incidents.csv.gz' # zipped because file is too big
population_file = './data/raw/population.csv'
crime_file = './data/raw/other_crime_annual.csv'
income_file = './data/raw/income.csv'
annual_file = './data/raw/annual_gun_deaths.csv'
alcohol_file = './data/raw/alcohol.csv'
provisions_file = './data/raw/provisions.csv'
election_file = './data/raw/election_results.csv'
registrations_file = './data/raw/registrations.csv'
substances_file = './data/raw/substances.csv'

daily_incidents_df = pd.read_csv(daily_incidents_file, parse_dates=True, compression='gzip')
population_df = pd.read_csv(population_file, parse_dates=True, index_col=0)
annual_gun_deaths_df = pd.read_csv(annual_file, parse_dates=True)
crime_df = pd.read_csv(crime_file, parse_dates=True)
income_df = pd.read_csv(income_file, parse_dates=True)
alcohol_df = pd.read_csv(alcohol_file, parse_dates=True)
provisions_df = pd.read_csv(provisions_file, parse_dates=True)
election_df = pd.read_csv(election_file)
registrations_df = pd.read_csv(registrations_file)
substances_df = pd.read_csv(substances_file)

***
## Incidents
### daily_incidents_df and population_df
Create the following DataFrames: 

    lat_long_df         Incidents with latitude and longitude coordinates (2014-2017)
    feature_df          Daily incidents per state, each with features from the previous year (2014-2017)
    by_date_total_df    Daily gun homicides per state indexed by date with states in columns

In [3]:
null_count = daily_incidents_df.isnull().sum()
null_count.sort_values()

incident_id                         0
date                                0
state                               0
city_or_county                      0
n_killed                            0
n_injured                           0
incident_url                        0
incident_url_fields_missing         0
incident_characteristics          327
source_url                        468
sources                           610
longitude                        7923
latitude                         7923
congressional_district          11945
address                         16497
participant_type                24864
participant_status              27627
state_senate_district           32336
participant_gender              36363
state_house_district            38773
participant_age_group           42120
notes                           81018
participant_age                 92299
n_guns_involved                 99452
gun_type                        99452
gun_stolen                      99499
participant_

There seems to be many missing values about the details of each incident. However, the 'state' and 'date' columns have no missing values, which is good for our objective (examining gun violence trends for each state).

In [4]:
# First let's make some columns that we'll need
daily_incidents_df['date'] = pd.to_datetime(daily_incidents_df['date']) # Turn the date into a datetime
daily_incidents_df['year'] = daily_incidents_df['date'].dt.year # Create a column for just the year

# Upon examining the data, it seems that records before 2014 have missing data, so we'll exclude them
daily_incidents_df = daily_incidents_df[daily_incidents_df['year'] >= 2014]

# Get number of casualties for each state, indexed by date
by_date_total_df = daily_incidents_df.groupby(['date', 'state'])['n_killed'].sum().unstack()
by_date_total_df = by_date_total_df.fillna(0) # Some days had no incidents (no entries). Fill with 0s

by_date_total_df.to_csv('./data/cleaned/by_date_total.csv')

In [5]:
# Get latitude and longitude information and save it
lat_long_df = daily_incidents_df[['state', 'latitude', 'longitude', 'n_killed']].dropna()
lat_long_df.to_csv('./data/cleaned/lat_long.csv')

In [6]:
# Create a feature DataFrame to store all of our features (to be used later in our model)
feature_df = by_date_total_df.resample('M').sum().stack().reset_index()
feature_df.columns = ['next_date', 'state', 'next_deaths']

# Add next year (the year which we want to predict on) and the previous year
feature_df['next_year'] = feature_df['next_date'].apply(lambda x: x.year)
feature_df['this_year'] = feature_df['next_year'] - 1
feature_df['last_year'] = feature_df['next_year'] - 2

# Add population data from population_df to feature_df
get_population = lambda x: population_df.loc[x['state'], str(x['this_year'])]
feature_df['population'] = feature_df[['state', 'this_year']].apply(get_population, axis=1)
    
# Reformat population data
population_df = population_df.stack().reset_index()
population_df.columns = ['state', 'year', 'population']
population_df['year'] = population_df['year'].astype(int)

# Put observations in each row for annual_gun_deaths_df; data is from 1999-2013
annual_1999_2013_df = annual_gun_deaths_df.set_index('state').stack().reset_index()
annual_1999_2013_df.columns = ['state', 'year', 'gun_deaths']

# Sum incidents annually and make each row an observation w/ state, year, and gun_deaths 
annual_2014_2017 = by_date_total_df[:'2017'].resample('A').sum().stack().reset_index()
annual_2014_2017['date'] = annual_2014_2017['date'].apply(lambda x: x.year)

# Rearrange columns and concat the dataframes
annual_2014_2017.columns = ['year', 'state', 'gun_deaths']
annual_2014_2017 = annual_2014_2017[['state', 'year', 'gun_deaths']]

# Note: later, we lose year 1999 when we merge with population since population_df goes from 2000-2017 only
annual_2000_2017_df = pd.concat([annual_1999_2013_df, annual_2014_2017])
annual_2000_2017_df['year'] = annual_2000_2017_df['year'].astype(int)

# Add population data to annual_2000_2017
annual_2000_2017_df = pd.merge(annual_2000_2017_df, population_df, how='left')
annual_2000_2017_df = annual_2000_2017_df.sort_values(['state', 'year'])

***
## Crime
### crime_df 
### annual_gun_deaths_df

We update `annual_2000_2017_df` and `feature_df` with crime data.

In [7]:
# Note: other_crime here is defined by crimes not including murder
crime_df = crime_df.drop(['population', 'index'], axis=1)
crime_df['year'] = crime_df['year'].astype(int)

# Update our feature dataframe with the crime rate for this_year and last_year (so we can see difference)
feature_df = pd.merge(feature_df, crime_df, left_on=['state','this_year'], 
                                            right_on=['state', 'year'])
feature_df = feature_df.drop('year', axis=1)

feature_df = pd.merge(feature_df, crime_df, left_on=['state','last_year'], 
                                            right_on=['state', 'year'], suffixes=('', '_old'))
feature_df = feature_df.drop('year', axis=1)

# Add the difference in violent crime as a feature
violent_crime_diff = feature_df['violent_crime'] - feature_df['violent_crime_old']
feature_df['violent_crime_diff'] = violent_crime_diff

crime_df['other_crime'] = crime_df[['rape_crime', 'robbery_crime', 'assault_crime', 
                                'burglary_crime', 'larceny_theft_crime', 'vehicle_theft_crime']].sum(axis=1)

annual_2000_2017_df = pd.merge(annual_2000_2017_df, crime_df, how='left')

***
## Personal Income
### income_df
We update `annual_2000_2017_df` and `feature_df` with personal income data.

In [8]:
# Add income data
income_by_state_df = income_df.set_index('state').stack().reset_index()
income_by_state_df.columns = ['state', 'year', 'income']
income_by_state_df['year'] = income_by_state_df['year'].astype(int)

annual_2000_2017_df = pd.merge(annual_2000_2017_df, income_by_state_df, how='left')
feature_df = pd.merge(feature_df, income_by_state_df, left_on=['state', 'this_year'], 
                                                      right_on=['state', 'year'])
feature_df = feature_df.drop('year', axis=1)

***
## Alcohol
### alcohol_df
Goals:

    annual_2000_2017_df    update with avg total alcohol consumption 
    feature_df              update with all alcohol features 

In [9]:
# Merge the alcohol features into the annual and feature data frames
annual_2000_2017_df = pd.merge(annual_2000_2017_df, alcohol_df, how='left')
feature_df = pd.merge(feature_df, alcohol_df, left_on=['state', 'this_year'], 
                                              right_on=['state', 'year'])
feature_df = feature_df.drop('year', axis=1)

***
## Provisions
### provisions_df
There are a lot of provisions, so we will selectively add them to the DataFrames later when we need them. 
For now, we update `annual_2000_2017_df` with the total number of provisions.

In [10]:
columns = ['state', 'year', 'lawtotal']
annual_2000_2017_df = pd.merge(annual_2000_2017_df, provisions_df[columns], how='left')

***
## Election Results
### election_df
We update `annual_2000_2017_df` and `feature_df` with election results data.

In [11]:
# For simplicity, let's expand the election_df s.t. it has entries for non-election years with values
# from the previous election. This way, we can just merge them with the other dfs later
election_expand = []
for i in range(4):
    temp_df = election_df.copy()
    temp_df['year'] = election_df['year'] + i
    election_expand.append(temp_df)

election_df = pd.concat(election_expand)

# Update with the election information
annual_2000_2017_df = pd.merge(annual_2000_2017_df, election_df, how='left')

feature_df = pd.merge(feature_df, election_df, left_on=['state', 'this_year'], right_on=['state', 'year'])
feature_df = feature_df.drop('year', axis=1)

***
## Registrations
### registrations_df
We update `annual_2000_2017_df` and `feature_df` with gun registrations data.

In [12]:
# Update the annual_2000_2017_df with the registrations
annual_2000_2017_df = pd.merge(annual_2000_2017_df, registrations_df, how='left')

# Put gun registration data
feature_df = pd.merge(feature_df, registrations_df, left_on=['state', 'this_year'], 
                                                    right_on=['state', 'year'])

feature_df = feature_df = feature_df.drop('year', axis=1)

feature_df = pd.merge(feature_df, registrations_df, left_on=['state','last_year'], 
                                                    right_on=['state', 'year'], suffixes=('', '_old'))
feature_df = feature_df.drop('year', axis=1)

# Add the change from last year to this year
registration_diff = feature_df['total_weapons'] - feature_df['total_weapons_old']
feature_df['registration_diff'] = registration_diff

***
## Substances
### substances_df
We update `annual_2000_2017_df` and `feature_df` with substances data.

In [13]:
# Update the annual_2000_2017_df with the substances data
annual_2000_2017_df = pd.merge(annual_2000_2017_df, substances_df, how='left')

# Put substances data in the feature_df
feature_df = pd.merge(feature_df, substances_df, left_on=['state', 'this_year'], 
                                                    right_on=['state', 'year'])

feature_df = feature_df = feature_df.drop('year', axis=1)

feature_df = pd.merge(feature_df, substances_df, left_on=['state','last_year'], 
                                                    right_on=['state', 'year'], suffixes=('', '_old'))

# Drop extra columns
feature_df = feature_df.drop('year', axis=1)
substances_df = substances_df.drop(['year', 'state'], axis=1)

# Add the change from last year to this year for each substance
for column in substances_df.columns:
    column_diff = feature_df[column] - feature_df[column + '_old']
    feature_df[column + '_diff'] = column_diff

## Wrapping up
Add a column to `annual_2000_2017_df` for normalized crime and gun homicides

Add the percent change from this year to the next year on each row of `feature_df`

In [14]:
# Add features normalized by population to the annual dataframe
annual_population = annual_2000_2017_df['population']
annual_2000_2017_df['gun_deaths_norm'] = annual_2000_2017_df['gun_deaths'] / annual_population * 100000
annual_2000_2017_df['other_crime_norm'] = annual_2000_2017_df['other_crime'] / annual_population * 100000

In [15]:
# For our predictions, we will be using the change in gun violence from the previous month. 
# Let's add that feature here:

# Calculate the change for each state for deaths 
states = by_date_total_df.columns

this_deaths = []
this_dates = []
for state in states:
    deaths = feature_df[feature_df['state'] == state]['next_deaths']
    this_deaths.append(deaths.shift())
    this_dates.append(feature_df[feature_df['state'] == state]['next_date'].shift())

# We use the previous month's violence rate as well as this month's date
feature_df['this_deaths'] = pd.concat(this_deaths)
feature_df['this_date'] = pd.concat(this_dates)

# Add the rate change from this_year to next_year
this_deaths = feature_df['this_deaths']
next_deaths = feature_df['next_deaths']
rate_change = (next_deaths - this_deaths) / np.clip(this_deaths, 1, None) # Fill zero values
feature_df['rate_change'] = rate_change

In [16]:
# Save annual and feature DataFrames
annual_2000_2017_df.to_csv('./data/cleaned/annual.csv')
feature_df.to_csv('./data/cleaned/feature.csv')

To recap, here are the DataFrames that were saved:

    lat_long_df                Incidents with latitude and longitude coordinates (2014-2017)
    by_date_tot_df             Gun homicides aggregated daily and by state (2014-2017)
    annual_2000_2017           Annual gun homicides with all other annual features (2000-2017)
    feature_df                 Gun homicides aggregated monthly and by state, paired with annual features from
                               the previous year (2014-2017)